In [1]:
import cv2
from matplotlib import pyplot as plt
import os
import torch
import numpy as np

In [14]:
def qr_code_detector(image_path, output_dir, save_annotated=True):
    """
    Detect QR codes using OpenCV and save results in YOLO format.

    Args:
        image_path (str): Path to the input image.
        output_dir (str): Directory to save results (images and labels).
        save_annotated (bool): If True, save the annotated image with QR code detections.
    """
    # Create output directories for images and labels
    os.makedirs(f"{output_dir}/images", exist_ok=True)
    os.makedirs(f"{output_dir}/labels", exist_ok=True)
    
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Error: Could not load image at {image_path}")
    h, w = image.shape[:2]
    image_name = os.path.splitext(os.path.basename(image_path))[0]  # Extract name without extension
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize QR Code detector
    qr_detector = cv2.QRCodeDetector()

    # Detect and decode QR codes
    data, _, vertices, _ = qr_detector.detectAndDecodeMulti(gray_image)

    labels = []  # Store YOLO-format labels
    if vertices is not None:
        for i, points in enumerate(vertices):
            # Calculate bounding box center, width, and height
            x_min = points[:, 0].min()
            y_min = points[:, 1].min()
            x_max = points[:, 0].max()
            y_max = points[:, 1].max()

            x_center = (x_min + x_max) / 2 / w  # Normalize by image width
            y_center = (y_min + y_max) / 2 / h  # Normalize by image height
            width = (x_max - x_min) / w         # Normalize width
            height = (y_max - y_min) / h        # Normalize height

            labels.append([0, x_center, y_center, width, height])  # Assuming class_id = 0 for QR codes

            # Draw bounding box on the image
            if save_annotated:
                points = points.astype(int)
                cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
                cv2.putText(image, f"QR {i}", tuple(points[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    else:
        print(f"No QR codes detected in {image_path}")

    # Save YOLO labels
    label_path = f"{output_dir}/labels/{image_name}.txt"
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")
    print(f"Saved YOLO label file: {label_path}")

    # Save annotated image
    if save_annotated:
        annotated_image_path = f"{output_dir}/images/{image_name}_result.jpg"
        cv2.imwrite(annotated_image_path, image)
        print(f"Saved annotated image: {annotated_image_path}")

# Example Usage
test_dir = "../data/test/samples/"
cv2_results_dir = "../data/test/results/cv2_detector/"

# Run QR code detection and save results for each image in the test directory
for filename in os.listdir(test_dir):
    if filename.endswith((".jpg", ".png", ".jpeg",".JPG")) and "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        print(f"Processing: {filepath}")
        qr_code_detector(filepath, cv2_results_dir, save_annotated=True)

test_light_130.JPG
Processing: ../data/test/samples/test_light_130.JPG
here
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_130.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_130_result.jpg
test_light_120.JPG
Processing: ../data/test/samples/test_light_120.JPG
here
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_light_120.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_light_120_result.jpg
test_110.JPG
Processing: ../data/test/samples/test_110.JPG
here
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_110.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_110_result.jpg
test_60.JPG
Processing: ../data/test/samples/test_60.JPG
here
Saved YOLO label file: ../data/test/results/cv2_detector//labels/test_60.txt
Saved annotated image: ../data/test/results/cv2_detector//images/test_60_result.jpg
test_light_60.JPG
Processing: ../data/test/sampl

In [ ]:
cv2_results_dir = "../data/test/results/yolov5n"

def save_yolo_results(model, image_path, output_dir):
    """
    Perform inference with YOLOv5 and save the results as annotated images and YOLO labels.

    Args:
        model: Loaded YOLOv5 model.
        image_path (str): Path to the input image.
        output_dir (str): Directory to save results (images and labels).
    """
    # Create output directories for images and labels
    os.makedirs(f"{output_dir}/images", exist_ok=True)
    os.makedirs(f"{output_dir}/labels", exist_ok=True)
    
    # Load the image
    image = cv2.imread(image_path)
    image_name = os.path.splitext(os.path.basename(image_path))[0]  # Extract name without extension
    
    # Perform inference
    results = model(image)

    # Save annotated image
    rendered_image = results.render()[0]  # Renders the detections
    rendered_image = cv2.cvtColor(rendered_image, cv2.COLOR_RGB2BGR)  # Convert to BGR
    rendered_image = np.ascontiguousarray(rendered_image, dtype=np.uint8)  # Ensure data type and memory

    # Save annotated image
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    annotated_image_path = f"{output_dir}/images/{image_name}_result.jpg"
    cv2.imwrite(annotated_image_path, rendered_image)
    print(f"Saved annotated image: {annotated_image_path}")
    
    # Save detection labels in YOLO format
    label_path = f"{output_dir}/labels/{image_name}.txt"
    detections = results.pandas().xywh[0]  # Get results as DataFrame
    
    with open(label_path, "w") as f:
        for _, row in detections.iterrows():
            class_id = row['class']
            x_center = row['xcenter'] / image.shape[1]  # Normalize x center
            y_center = row['ycenter'] / image.shape[0]  # Normalize y center
            width = row['width'] / image.shape[1]       # Normalize width
            height = row['height'] / image.shape[0]     # Normalize height
            
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    
    print(f"Saved YOLO label file: {label_path}")

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/yolov5n_best.pt')

# Verify the model is loaded
print("Model loaded successfully!")

# Input and output paths
image_path = '../data/test/test_60.JPG'
output_dir = "../results/yolov5n"

# Run inference and save results
for filename in os.listdir(test_dir):
    if "out" not in filename and filename[0] != ".":
        filepath = os.path.join(test_dir, filename)
        print(filepath)
        save_yolo_results(model, filepath, output_dir)

Using cache found in /Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-12 Python-3.11.11 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


Model loaded successfully!
../data/test/samples/test_light_130.JPG


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved annotated image: ../results/yolov5n/images/test_light_130_result.jpg
Saved YOLO label file: ../results/yolov5n/labels/test_light_130.txt
../data/test/samples/test_light_120.JPG


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Saved annotated image: ../results/yolov5n/images/test_light_110_result.jpg
Saved YOLO label file: ../results/yolov5n/labels/test_light_110.txt
../data/test/samples/test_140.JPG


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved annotated image: ../results/yolov5n/images/test_140_result.jpg
Saved YOLO label file: ../results/yolov5n/labels/test_140.txt
../data/test/samples/test_light_100.JPG


/Users/aq_home/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved annotated image: ../results/yolov5n/images/test_light_100_result.jpg
Saved YOLO label file: ../results/yolov5n/labels/test_light_100.txt
